Para criar uma classe que realiza a otimização de ângulos de portas paramétricas usando princípios SOLID, podemos seguir alguns passos básicos:

Definir uma interface que represente a otimização de ângulos. Isso pode ser feito usando a abstração fornecida pelo princípio "I" de SOLID (Interface Segregation Principle). A interface deve definir os métodos necessários para realizar a otimização, como optimize_angles() e get_optimized_circuit().

In [ ]:
from abc import ABC, abstractmethod
from braket.circuits import Circuit

class AngleOptimizer(ABC):
    @abstractmethod
    def optimize_angles(self, circuit: Circuit, operator: object) -> None:
        pass
    
    @abstractmethod
    def get_optimized_circuit(self) -> Circuit:
        pass


Implementar uma classe que realiza a otimização de ângulos usando um algoritmo de otimização apropriado, como o método de gradiente conjugado. Isso pode ser feito usando a abstração fornecida pelo princípio "D" de SOLID (Dependency Inversion Principle), que estipula que as classes devem depender de abstrações, não de implementações concretas.

In [ ]:
from braket.circuits import Circuit
from scipy.optimize import minimize
from typing import List
from .angle_optimizer import AngleOptimizer

class CGAngleOptimizer(AngleOptimizer):
    def __init__(self, maxiter: int = 1000, tol: float = 1e-6) -> None:
        self.maxiter = maxiter
        self.tol = tol
        self._optimized_circuit = None
    
    def optimize_angles(self, circuit: Circuit, operator: object) -> None:
        def cost_function(x: List[float]) -> float:
            # Define a função de custo que é a norma do operador desejado menos o operador calculado
            op = operator(x)
            return abs(op - target_op).max()

        # Define o alvo como o operador desejado
        target_op = operator(circuit.params)
        # Otimiza a função de custo
        result = minimize(cost_function, circuit.params, method="CG", tol=self.tol, options={"maxiter": self.maxiter})
        # Atualiza os ângulos no circuito
        circuit.params = result.x
        # Define o circuito otimizado
        self._optimized_circuit = circuit
    
    def get_optimized_circuit(self) -> Circuit:
        if self._optimized_circuit is None:
            raise ValueError("No circuit has been optimized yet.")
        return self._optimized_circuit


Neste exemplo, implementamos a classe CGAngleOptimizer, que usa o método de gradiente conjugado para otimizar os ângulos do circuito. A classe herda da interface AngleOptimizer e implementa os métodos optimize_angles() e get_optimized_circuit(). O método optimize_angles() define a função de custo e usa o método de otimização minimize() da biblioteca SciPy para otimizar os ângulos. O método get_optimized_circuit() retorna o circuito otimizado.

Com essas classes definidas, podemos usá-las da seguinte maneira:

In [ ]:
from braket.circuits import Circuit
from .cg_angle_optimizer import CGAngleOptimizer

# Define um circuito e um operador a ser otimizado
circ = Circuit().rx("theta", 0).


USANDO ADAM


In [ ]:
from braket.circuits import Circuit
from scipy.optimize import minimize
from typing import List
from .angle_optimizer import AngleOptimizer

class AdamAngleOptimizer(AngleOptimizer):
    def __init__(self, maxiter: int = 1000, tol: float = 1e-6, learning_rate: float = 0.1) -> None:
        self.maxiter = maxiter
        self.tol = tol
        self.learning_rate = learning_rate
        self._optimized_circuit = None
    
    def optimize_angles(self, circuit: Circuit, operator: object) -> None:
        def cost_function(x: List[float]) -> float:
            # Define a função de custo que é a norma do operador desejado menos o operador calculado
            op = operator(x)
            return abs(op - target_op).max()

        # Define o alvo como o operador desejado
        target_op = operator(circuit.params)
        # Otimiza a função de custo
        result = minimize(cost_function, circuit.params, method="L-BFGS-B", tol=self.tol, options={"maxiter": self.maxiter}, jac=True)
        # Atualiza os ângulos no circuito
        circuit.params = result.x
        # Define o circuito otimizado
        self._optimized_circuit = circuit
    
    def get_optimized_circuit(self) -> Circuit:
        if self._optimized_circuit is None:
            raise ValueError("No circuit has been optimized yet.")
        return self._optimized_circuit
